In [1]:
import numpy as np
import pandas as pd

raw_data = open('TWITTER-Real-Graph-Partial.txt','r')
preprocessed_data = open('preprocessed_data.txt','w')
for i in raw_data:
    if i.startswith('e'):
        if i[3] != ' ':
            preprocessed_data.write(i[0:4] + ',' + i[5:])
        else:
            preprocessed_data.write(i[0:3] + ',' + i[4:])
    else:
        preprocessed_data.write(i)
raw_data.close()
preprocessed_data.close()

file = pd.read_csv('preprocessed_data.txt', sep=' ', header = None)

nodes_name = pd.Series(pd.unique(file[file[0] == 'n'][2]))
all_nodes = list(file[file[0] == 'n'][2])
nodes_name_frq = {}

for i in nodes_name:
    nodes_name_frq[i] = all_nodes.count(i) / len(all_nodes)

In [4]:
def BuildInput(graph_raw):
    nodes_num = graph_raw[graph_raw[0] == 'n'].shape[0]
    edges = graph_raw[graph_raw[0] == 'e'][1]
    weights = graph_raw[graph_raw[0] == 'e'][2]
    features = graph_raw[graph_raw[0] == 'n'][2]
    label = int(graph_raw[graph_raw[0] == 'x'][1])
    
    if label == -1:
        label = np.array([0,1])
    else:
        label = np.array([1,0])

    adj_matrix = np.zeros((14,14))  #need to find maximum dimension
    weighted_adj_matrix = np.zeros([14,14])
    feature_matrix = np.zeros([14,1])

    for index in edges.index:
        if edges[index][1] != ',':
            row = int(edges[index][0:2])
            column = int(edges[index][3:])
        else:
            row = int(edges[index][0:1])
            column = int(edges[index][2:])
        weighted_adj_matrix[row-1,column-1] = weights[index]
        weighted_adj_matrix[column-1,row-1] = weights[index]
        adj_matrix[column-1,row-1] = 1
        adj_matrix[row-1,column-1] = 1
        
    
    for i,word in enumerate(features):
        adj_matrix[i,i] = 1
        weighted_adj_matrix[i,i] = 1/len(all_nodes)
        feature_matrix[i,0] = nodes_name_frq[word]
        
    d_matrix = np.diag(np.array(np.sum(adj_matrix,0)))    
    for i,word in enumerate(features):
        d_matrix[i,i] = 1/d_matrix[i,i]
        
    return feature_matrix,weighted_adj_matrix,adj_matrix,d_matrix,label

In [5]:
graph_seg_head = 0
count = 0
sample_num = len(file[file[0] == 'x'].index)
w_adj_matrix_whole = np.ndarray(shape=(sample_num,14,14))
adj_matrix_whole = np.ndarray(shape=(sample_num,14,14))
label_whole = np.zeros([sample_num,2])
fea_matrix_whole = np.ndarray(shape=(sample_num,14,1))
d_matrix_whole = np.ndarray(shape=(sample_num,14,14))

for n,graph_seg_end in enumerate(file[file[0] == 'x'].index):
    fea_matrix,w_adj_matrix,adj_matrix,d_matrix,label = BuildInput(file.loc[graph_seg_head:graph_seg_end,:])
    adj_matrix_whole[n] = adj_matrix
    w_adj_matrix_whole[n] = w_adj_matrix
    label_whole[n] = label
    fea_matrix_whole[n] = fea_matrix
    d_matrix_whole[n] = d_matrix
    graph_seg_head = graph_seg_end + 1
    count = count + 1

    if (count%10000 == 0):
        print('Comuptation complete:',count,'/',sample_num)



np.save('adj_matrix.npy',adj_matrix_whole)
np.save('label.npy',label_whole)
np.save('w_adj_matrix.npy',w_adj_matrix_whole)
np.save('fea_matrix.npy',fea_matrix_whole)
np.save('d_matrix.npy',d_matrix_whole)

print('Completed!')

Comuptation complete: 10000 / 144033
Comuptation complete: 20000 / 144033
Comuptation complete: 30000 / 144033
Comuptation complete: 40000 / 144033
Comuptation complete: 50000 / 144033
Comuptation complete: 60000 / 144033
Comuptation complete: 70000 / 144033
Comuptation complete: 80000 / 144033
Comuptation complete: 90000 / 144033
Comuptation complete: 100000 / 144033
Comuptation complete: 110000 / 144033
Comuptation complete: 120000 / 144033
Comuptation complete: 130000 / 144033
Comuptation complete: 140000 / 144033
Completed!


In [4]:
file

,0,1,2
0,n,1,apple
1,n,2,store
2,n,3,buy
3,n,4,mac
4,e,"1,2",3.034013E-4
5,e,"3,4",1.6500772E-4
6,e,"3,2",3.5130675E-4
7,g,2191352508,56
8,x,1,NaN
9,n,1,move


{'apple': 0.00021523224420078241,
 'store': 0.001029671056256543,
 'buy': 0.004421731224860874,
 'mac': 0.00039086175546862087,
 'move': 0.002415766708909582,
 'dad': 0.002083448123863574,
 'butter': 9.2980329494738e-05,
 'peanut': 0.00011708634084522563,
 'fan': 0.0022022563226624056,
 'laker': 0.0005475508292467904,
 'die': 0.0017958978456113284,
 'tummy': 0.00021867596010799494,
 'hurt': 0.0027050388451154333,
 'feel': 0.01236810568075376,
 'museum': 0.00015496721582456335,
 'night': 0.013943605708303488,
 'rest': 0.002033514243208992,
 'day': 0.03311305030580197,
 'book': 0.0028324563336822968,
 'mention': 0.0002996032839274891,
 'dont': 0.001980136646647198,
 'follow': 0.00428570444652598,
 'reply': 0.001813116425147391,
 'sunday': 0.0022401371976417434,
 'haha': 0.008976045512149429,
 'realize': 0.0015358972946167833,
 'life': 0.005613256928756406,
 'people': 0.007118160780208276,
 'guess': 0.004020538321670615,
 'matter': 0.000278940988484214,
 'reach': 0.0001188081987988319,
 '